# Reading and Writing Back to an S3 Bucket

This image shows the originals and the files that were written back using the code in this notebook.

![](/Workspace/Users/joy.garnett@databricks.com/Customer Questions/FNMA/ETL Logging Native, Glue, and Redshift/images/S3-direct-read-write.png)

## Directly Connecting to a CSV in an S3 bucket

In [0]:
# Reading

df = spark.read.format("csv").option("header", True).load("s3a://jgarnett-us-east-1-bucket/s3_direct/MNQ_Stock.csv")
display(df.limit(5))

Trade #,Type,Date/Time,Signal,Price USD,Position size (qty),Position size (value),Net P&L USD,Net P&L %,Run-up USD,Run-up %,Drawdown USD,Drawdown %,Cumulative P&L USD,Cumulative P&L %
1,Exit long,2025-08-31 19:13,Short,23535.75,5,117553.75,246.25,0.10,350,0.15,-5,0.00,246.25,0.49
1,Entry long,2025-08-31 18:39,Long,23510.75,5,117553.75,246.25,0.10,350,0.15,-5,0.00,246.25,0.49
2,Exit short,2025-08-31 19:46,Long,23535.5,5,117678.75,0,0.00,66.25,0.03,-66.25,-0.03,246.25,0.49
2,Entry short,2025-08-31 19:13,Short,23535.75,5,117678.75,0,0.00,66.25,0.03,-66.25,-0.03,246.25,0.49
3,Exit long,2025-08-31 19:49,Short,23533.75,5,117677.5,-20,-0.01,21.25,0.01,-68.75,-0.03,226.25,0.45


In [0]:
# Write back to the S3 bucket
prefix = "renamed_and_saved_"
new_cols = [f'{prefix}({c})' for c in df.columns]

df_renamed = df.toDF(*new_cols)  # rename all columns in one step


tmp_path = "dbfs:/tmp/MNQ_Stock_single_tmp"
final_path = "s3a://jgarnett-us-east-1-bucket/s3_direct/MNQ_Stock_Renamed_and_Saved.csv"

#1 Writing as a single part folder
df_renamed.write.mode("overwrite").option('header', 'true').csv(tmp_path)

#2 Find single part file
files = dbutils.fs.ls(tmp_path)
part_file = [f.path for f in files if f.name.startswith("part-")][0]

#3 Copy to final S3 file and clean up

dbutils.fs.cp(part_file, final_path)
dbutils.fs.rm(tmp_path, recurse=True)


True

## Directly Connecting to a JSON File in an S3 Bucket

In [0]:
# Reading from a json in S3

df2 = spark.read.format("json").option("multiLine", True).load("s3a://jgarnett-us-east-1-bucket/s3_direct/zoneinfo_data.json")
display(df2)

data metadata List(List({Wp48S^xk9=GL@E0stWa761SMbT8$j-~f{VGF<>F7KxBg5R*{Ksocg8-YYVul=v7vZzaHN, uC=da5UI2rH18c!OnjV{y4u(+A!!VBKmY&$ORw>7UO^(500B;v0RR91bXh%WvBYQl0ssI2, 00dcD), List({Wp48S^xk9=GL@E0stWa761SMbT8$j;0b&Kz+C_;7KxBg5R*{N&yjMUR~;C-fDaSOU;q-~, FqW+4{YBjbcw}`a!dW>b)R2-0a+uwf`P3{_Y@HuCz}S$J$ZJ>R_V<~|Fk>sgX4=%0vUrh-, lt@YP^Wrus;j?`Th#xRPzf<<~Hp4DH^gZX>d{+WOp~HNu8!{uWu}&XphAd{j1;rB4|9?R!, pqruAFUMt8#*WcrVS{;kLlY(cJRV$w?d2car%R s>q9BgTU4, Ht-tQKZ7Z`9QqOb?R#b%z?rk>!CkH7jy3wja4NG2q)H}fNRKg8v{);Em;K3Cncf4C6&Oaj, V+DbX%o4+)CV3+e!Lm6dutu(0BQpH1T?W(~cQtKV*^_Pdx!LirjpTs?Bmt@vktjLq4;)O!, rrly=c*rwTwMJFd0I57`hgkc?=nyI4RZf9W$6DCWugmf&)wk^tWH17owj=#PGH7Xv-?9$j, njwDlkOE+BFNR9YXEmBpO;rqEw=e2IR-8^(W;8ma?M3JVd($2T>IW+0tk|Gm8>ftukRQ9J, 8k3brzqMnVyjsLI-CKneFa)Lxvp_a q40f}0J3VVoWL5rox, `Kptivcp}o5xA^@>qNI%?zo=Yj4AMV?kbAA)j(1%)+Pp)bSn+7Yk`M{oE}L-Z!G6 OMr5G+h, p)$3Lg{ono{4cN>Vr&>L4kXH;_VnBL5U!LgzqE%P7QQ* tue}O`3(TZ0`aKn&~8trOQ-rBXCp)f@P6RMO4l0+;b|5-pk9_ryNh}Zc*v%mvz_#, yd 6fjB0g9{MmMnu8bG%#C~ugXK^S^k@?ab#, O|aE>dDTt4s4n69(~@t~!wniV%g 7khFx~I*4>Y|V$4j5%KPF*-FyKIi@!Ho&, x8QQsksYt8)D+W)Ni!=G`ogSu^vLL-l#7A7=iIAKL2SuZk9F}NfNk86VI)9WZE?%2wC-ya, F~z#Qsq)LH0|_D8^5fU8X%GeQ4TB>R-dlziA&tZe&1ada208!$nk`7bOFO2S00G `Z@1A~t&lyL{p{eM{5)QGf7Mo5FW9==mlyXJt2, UwpntR7H0eSq!(aYq#aqUz&RM*tvuMI)AsM?K3-dV3-TT{t)!Iy#JTo=tXkzAM9~j2YbiO, ls3(H8Dc>Y|D1aqL51vjLbpYG;GvGTQB4bXuJ%mA;(B4eUpu$$@zv2vVcq-Y)VKbzp^tei, uzy}R{Luv DjpuVb`79O+CBmg{Wx!bvx$eu4zRE&, PehMb=&G<9$>iZ|bFE)0=4I?KLFGBC0I(0_svgw0%FiMsT%koo*!nEYc6GY@QnU}&4Isg;, l=|khi(!VaiSE2=Ny`&&tpi~~;{$u N}%f|7mBhAy; s3YT^sy!$eG~?`9mNJC9@4Bac_p^BZh)Yd_rWW5qh-?tKY(>5VHO, L*iT8P@wCavLj^yYbnDR+4ukhS+xPrpl)iqB?u)bj9a2aW==g6G3lCJd>(+Blf r)~^40F4f>cRZ^UF;RibfZ>0m73hR, C{$vTfC(STN`g7(B<=Z2556{}0`?p&|Akkst!4Xy4OT;A@c$XTUI3FRRjy*KA7uC56FD)z, ^X{WV*sr(w!c$W357o!&eLO2wTDNOyw@gf(&R< t;=-Tu1TV{>%8ZVATC9tjD8|(&`$9YHvZ9bVe#>w, |8c;Tg|xE&)`*}LwM*E}q}q8^Qja%p`_U)*5DdLI9O@!e=3jFjOCrCq28b_bb;s>%D#iJB, CWJi{JH!Js;6nfayos$kq^OEX00HO-lokL0!mqm{vBYQl0ssI200dcD), List({Wp48S^xk9=GL@E0stWa761SMbT8$j;0fRZ<6QtM7KxBg84(fsEAUJ$J{f-TXlPEUec5Ee, n+hsD4lC(QYax=JdSpoyje8%VM`GW} {bJ8@y$A8O&*$pw{(f~Os#}2w, eX6^Rgi$IT%n^V^85L>$_c7{cB^#ogV=rHBJGiz-RQNFGK?gdPi|q)j`&8)}KJ{qo6dixa, 9@yYyVg+%lo0nO+Tw0-w2hJ%mafy WL)|, ) ?W6Bi%FWuGPA1Dru$XR4SZANsAthU2EoKH F6oEtKq`rwP, (VNegnI_NI%;ma$)wj{k!@KFB30Yo)IOr l>)$)D|+(5h&+%2vuwGuy^@S8FT^s21V5};>VA9Iu;?8bHz#r<;JtfZDI1(FT@edh0#, MYW$A1qkMGIwTZqqdYNE3gl#zp&NbL9Mp=voqN|;?gqR&4$)1`znddtEyuKS*^nMMD=0^>, 7^z6-C4P67UWOXuMBubP>j6i~03aR@jD^-Y`JSYu#Yp0P8dLLJ0QOPE8=BoiuRX59YW7xg, WiexjHX%&0?`ZQCdxCdL^qd1v@kOjQKaWo2Y1++~LcA%FTq?5o<?(jL(_Uo}I}k_Fwflcr, aovwSR_(ILA6li %}fX1-RIvlB)1#iTNomGnUL=nM!>Ix|AGtON7!F1O?53kqlC2o-`ZGw*+s, NM$^9znsIJMwlgscE`|O3|;BRgsQMYm~`uv+nvuv`nigRa}X=BX=A5Sw$)WEklF7&c>_~$, zJ(m--bqXgiN^w-U=BJH9C0Qro(x90zo@rK;&TJ$nI@&k$ORgOb2 s%gWbc}ok_27)Eoku~Fq|B-Ps+4J_, HPJMLJ2^_)cOU$p&3kNAlrV!)%~6r$BJ>OOi~=-<6byle{?zd4J{NG}o8tw|+#ZNLcpNwk, TuPE~sbJB8_RZb2DopStO+Wwux~F#S59zm%00I98;S&G=b(j+6vBYQl0ssI200dcD), List({Wp48S^xk9=GL@E0stWa761SMbT8$j-~luMgIxeB7KxBg5R*;y?l4Rl4neXH3cv!OtfK@h, KZzauI)S!FSDREPhhBS6Fb$&Vv#7%;?Te|>pF^0HBr&z_Tk<%vMW_QqjevRZOp8XVFgP<8, TkT#`9H&0Ua;gT1#rZLV0HqbAKK;_z@nO;6t0L }hOdk<>TdUa07R(LPI6@!GU$ty4=mwqHG-XVe*n(Yvgdlr+FqIU18!osi)48t~eWX8)&L, G)Ud^0zz@*AF+2r7E}N f9Y72K~o-T%}D&z%}#7g 2br?oH6ZiYH^%>J3D)TPKV(JY*bwjuw5=DsPB@~CrR OZeN, x>A*H&CHrWt0`EP`m!F%waepl#|w#&`XgVc?~2M3uw$fGX~tf_Il!q#Aa<*8xlzQ2+7r6Z, ^;Laa9F(WB_O&Dy2r>~@kSi16W{=6+i5GV=Uq~KX*~&HUN4oz7*O(gXIr}sDVcD`Ikgw#|, 50ssal8s)Qy;?YGCf;*UKKKN!T4!Kqy_G;7 PfQapugqvVBKy12v3TVH^L2, 0?#5*VP~MOYfe$h`*L!7@tiW|_^X1N%<}`7YahiUYtMu5XwmOf3?dr+@zXHwW`z}ZDqZlT, <2Cs(<1%M!i6o&VK89BY0J7HPIo;O62s=|IbV^@y$N&# =>i^F00FcHoDl#3, Mdv&xvBYQl0ssI200dcD), List({Wp48S^xk9=GL@E0stWa761SMbT8$j;0>b$_+0=h7KxBg5R*;&J77#T_U2R5sleVWFDmK~, Kzj5oh@` QKHvW^6V{jU-w>qg1tSt0c^vh;?qAqA0%t?;#S~6U8Qi, v&f1s9IH#g$m1k1a#3+lylw4mwT4QnEUUQdwg+xnEcBlgu31bAVabn41OMZVLGz6NDwG%X, uQar!b>GI{qSahE`AG}$kRWbuI~JCt;38)Xwbb~Qggs55t+MAHIxgDxzTJ;2xXx99+qCy4, 45kC#v_l8fx|G&jlVvaciR<-wwf22l%4(t@S6tnX39#_K(4S0fu$F

In [0]:
from pyspark.sql.functions import col

flat_cols = []
for c in df2.columns:
    # For nested fields like "parent.child", Spark shows them as "parent.child" in schema.
    # Select with alias to replace dots in the output name.
    flat_cols.append(col(c).alias(c.replace(".", "_")))

df2_flat = df2.select(*flat_cols)

# Rename all columns with a prefix
prefix = "renamed_and_saved_"
new_cols = [f'{prefix}({c})' for c in df2_flat.columns]
df2_renamed = df2_flat.toDF(*new_cols)  # rename all columns at once

In [0]:
# Writing to a json in S3
tmp_path2 = "dbfs:/tmp/my_json_single_tmp"
final_path = "s3a://jgarnett-us-east-1-bucket/s3_direct/renamed_zone_info.json"

# 1) Write to a temporary folder as a single-part JSON dataset
(
    df2_renamed
      .coalesce(1)              # force single output file
      .write
      .mode("overwrite")
      .json(tmp_path2)
)

# 2) Find the single part file
files = dbutils.fs.ls(tmp_path2)
part_file = [f.path for f in files if f.name.startswith("part-")][0]

# 3) Copy to final S3 key and clean up
dbutils.fs.cp(part_file, final_path)
dbutils.fs.rm(tmp_path2, recurse=True)



True